In [1]:
# Tools creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


In [2]:
# Used the inbuilt tools of Wikipedia

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool.name

'wikipedia'

In [4]:
# Used inbuilt tool: Arxiv

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [6]:
tools=[wiki_tool, arxiv_tool]


In [ ]:

## Create a Custom tool for RAG Tool 
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [11]:
loader = WebBaseLoader("https://docs.langsmith.com/")
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=splitter.split_documents(docs)[0:200]
vector_db=FAISS.from_documents(documents, OllamaEmbeddings())
retriver = vector_db.as_retriever()
retriver

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000260ACB34E50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))